[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/8k-earnings-release-python.ipynb)

# Extract 8-K Earnings Releases with Python -- Free, No API Key

Use **edgartools** to extract earnings releases from SEC 8-K filings -- completely free, no API key or paid subscription required. When companies report quarterly earnings, they file an 8-K with the press release containing income statements, balance sheets, and key metrics -- often hours before the data appears on financial sites.

**What you'll learn:**
- Detect which 8-K filings contain earnings data
- Extract income statements and balance sheets from earnings press releases
- Read the full press release text
- Browse all financial tables in an earnings release
- Scan recent 8-K filings to find earnings announcements
- Compare earnings data across companies

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
import pandas as pd
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Get Earnings Data from an 8-K in 3 Lines

When a company reports quarterly earnings, they file an 8-K containing the press release with financial tables. edgartools detects earnings filings automatically and parses the tables:

In [ ]:
filing = Company("AAPL").get_filings(form="8-K")[0]
eightk = filing.obj()

print(f"Company:       {eightk.company}")
print(f"Filed:         {eightk.filing_date}")
print(f"Report date:   {eightk.date_of_report}")
print(f"Items:         {eightk.items}")
print(f"Has earnings:  {eightk.has_earnings}")

eightk.get_income_statement()

## Balance Sheet from the Earnings Release

Most earnings press releases include a condensed balance sheet alongside the income statement:

In [ ]:
eightk.get_balance_sheet()

## Browse All Financial Tables

Earnings releases often contain more than just the main statements -- segment data, EPS reconciliations, and non-GAAP metrics. Browse all parsed tables:

In [ ]:
earnings = eightk.earnings

print(f"Scale:            {earnings.detected_scale}")
print(f"Financial tables: {len(earnings.financial_tables)}\n")

for i, table in enumerate(earnings.financial_tables):
    df = table.dataframe
    print(f"Table {i+1}: {table.statement_type.value}")
    print(f"  Rows: {df.shape[0]}, Periods: {table.periods}")

## Read the Full Press Release

The earnings press release is attached as an exhibit (typically EX-99.1). Access the full text for NLP analysis or to read management commentary:

In [ ]:
pr = eightk.press_releases[0]
text = pr.text()

print(f"Document: {pr.document}")
print(f"Length:   {len(text):,} characters\n")
print(text[:800])

## Access 8-K Items by Number

Every 8-K filing reports one or more numbered items. Item 2.02 ("Results of Operations and Financial Condition") signals an earnings announcement:

In [ ]:
for item in eightk.items:
    text = eightk[item]
    if text:
        print(f"{item}: {len(text):,} chars")

print(f"\nItem 2.02 text:\n")
print(eightk["2.02"][:500])

## Scan Recent 8-K Filings for Earnings

During earnings season, dozens of companies file 8-K reports each day. Scan recent filings to find earnings announcements:

In [ ]:
filings = get_filings(form="8-K")
rows = []

for f in filings.head(30):
    eightk = f.obj()
    if eightk.has_earnings:
        earnings = eightk.earnings
        rows.append({
            "Company": f.company[:30],
            "Filed": str(f.filing_date),
            "Tables": len(earnings.financial_tables),
            "Scale": str(earnings.detected_scale),
        })

print(f"Found {len(rows)} earnings filings in latest {len(filings):,} 8-K filings\n")
pd.DataFrame(rows)

## Compare Earnings Across Companies

Pull the latest earnings filing from multiple companies and compare what data is available:

In [ ]:
tickers = ["AAPL", "MSFT", "NVDA", "GOOG"]
rows = []

for ticker in tickers:
    for f in Company(ticker).get_filings(form="8-K").head(5):
        eightk = f.obj()
        if eightk.has_earnings:
            earnings = eightk.earnings
            rows.append({
                "Ticker": ticker,
                "Quarter": str(eightk.date_of_report),
                "Tables": len(earnings.financial_tables),
                "Scale": str(earnings.detected_scale),
                "Income Rows": eightk.get_income_statement().shape[0],
            })
            break

pd.DataFrame(rows).set_index("Ticker")

## Why EdgarTools?

EdgarTools is free and open-source. Compare extracting 8-K earnings data:

**With edgartools (free, no API key):**
```python
eightk = Company("AAPL").get_filings(form="8-K")[0].obj()
eightk.has_earnings              # Auto-detect earnings filings
eightk.get_income_statement()    # Parsed income statement
eightk.press_releases[0].text()  # Full press release text
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import ExtractorApi
api = ExtractorApi(api_key="YOUR_PAID_API_KEY")
# ... find 8-K URL, download exhibit, parse HTML tables manually,
# ... no automatic earnings detection or table classification
```

With edgartools, 8-K earnings data is automatically detected and parsed -- income statements, balance sheets, and press release text all accessible instantly.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# ── Parse an 8-K ──
filing = Company("AAPL").get_filings(form="8-K")[0]
eightk = filing.obj()

# ── Detect earnings ──
eightk.has_earnings                # True if Item 2.02 + parseable tables
eightk.has_press_release           # True if EX-99 exhibit exists
eightk.items                       # List of items: ['Item 2.02', 'Item 9.01']

# ── Financial tables ──
eightk.get_income_statement()      # Income statement as DataFrame
eightk.get_balance_sheet()         # Balance sheet as DataFrame
eightk.get_cash_flow_statement()   # Cash flow as DataFrame (if available)

# ── Earnings release object ──
earnings = eightk.earnings
earnings.financial_tables          # All parsed financial tables
earnings.detected_scale            # Scale: millions, billions, etc.
earnings.income_statement          # FinancialTable object

# ── Press release ──
pr = eightk.press_releases[0]
pr.text()                          # Full press release as text
pr.document                        # Exhibit filename

# ── Item text ──
eightk["2.02"]                     # Item 2.02 text
eightk["9.01"]                     # Item 9.01 text

# ── Scan for earnings ──
for f in get_filings(form="8-K").head(30):
    eightk = f.obj()
    if eightk.has_earnings:
        print(f.company, eightk.get_income_statement().shape)
```

## What's Next

You've learned how to extract earnings data from SEC 8-K filings. Here are related tutorials:

- [Parse 10-Q Quarterly Earnings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/10q-quarterly-earnings-python.ipynb)
- [Extract Financial Statements from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/financial-statements-sec-python.ipynb)
- [Extract Revenue and Earnings from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/extract-revenue-earnings-python.ipynb)
- [SEC EDGAR API in Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/sec-edgar-api-python.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*